In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from scipy.stats import pearsonr
from collections import Counter

# importamos as libs
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
import numpy as np
import tensorflow as tf
sns.set_palette("Set2")

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
# calcula metricas de regressao
def score_regression_metrics(y_test, y_test_pred):

    RMSE = mean_squared_error(y_true=y_test, y_pred=y_test_pred, squared=False)
    MAE = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    MAPE = mean_absolute_percentage_error(y_true=y_test, y_pred=y_test_pred)
    R2 = r2_score(y_true=y_test, y_pred=y_test_pred)

    scores = {
        "neg_root_mean_squared_error": RMSE,
        "neg_mean_absolute_error": MAE,
        "neg_mean_absolute_percentage_error": MAPE,
    }

    return scores

def show_scores (scores):

    print ("Scores obtidos:")

    print (f"RMSE: {scores['neg_root_mean_squared_error']}")
    print (f"MAE: {scores['neg_mean_absolute_error']}")
    print (f"MAPE: {scores['neg_mean_absolute_percentage_error']}")

In [3]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [4]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [5]:
def set_seeds():
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

In [216]:
df = pd.read_csv("input_and_output.csv")

In [217]:
SEED=47

TO_DROP = [
    "pib_pc",
    "idh_long",
    "nfsp",
    "estoque",
]

VARS = [
    'pib_pmc',
    'pib_pcpt',
    'populacao',
    'pib_cc',
    'ipca',
    'incc',
    'igp',
    'selic',
    'idh_renda',
    'idh_educacao',
    'desemprego'
]

batch_size = 32
epochs = 200
verbose = 1
timesteps = 3

In [218]:
# TO_DROP = [

# ]

# VARS = [
#     "pib_pc",
#     "idh_long",
#     "nfsp",
#     "estoque",
#     'pib_pmc',
#     'pib_pcpt',
#     'populacao',
#     'pib_cc',
#     'ipca',
#     'incc',
#     'igp',
#     'selic',
#     'idh_renda',
#     'idh_educacao',
#     'desemprego'
# ]

In [219]:
df = df.drop(TO_DROP, axis=1)

In [220]:
# for var in VARS:
#     df[var] = df[var].clip(upper=df[var].quantile(.75))
    

In [221]:
# for var in VARS:
#     df[var] = df[var].clip(upper=df[var].quantile(.75))
    

In [222]:
# df[VARS].hist(bins=25, figsize=(20, 20), layout=(-1, 5), edgecolor="black")
# plt.tight_layout()

In [223]:
df_sorted = df.sort_values(by=["estados", "data"])
for estado in df.estados.unique():
    indexes = df_sorted[df_sorted["estados"] == estado].index
    df_sorted.loc[indexes, "producao"] = df_sorted.loc[indexes].producao.shift(1)

In [224]:
df_sorted = df_sorted[df_sorted.producao.notna()]

In [240]:
df_sorted = df_sorted.sort_values(by=["data", "estados"])

In [241]:
df = df_sorted

In [210]:
x = df.drop(["data", "consumo"], axis=1).copy()

# processo de one-hot
x = pd.get_dummies(data=x, columns=["estados"], drop_first=True)

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [211]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.9611790983876279


In [212]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 40425.21197054409
MAE: 24012.83146913503
MAPE: 0.46357947133600264

TESTE
Scores obtidos:
RMSE: 35298.35252173542
MAE: 24843.3921823251
MAPE: 0.4208661889241405


In [234]:
x = df.drop(["data", "consumo"], axis=1).copy()

# processo de one-hot
x = pd.get_dummies(data=x, columns=["estados"], drop_first=True)

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [235]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.8169891627286769


In [236]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 32378.898120001966
MAE: 20618.38722036052
MAPE: 0.44035108655585387

TESTE
Scores obtidos:
RMSE: 177479.21877353307
MAE: 108770.17895378474
MAPE: 0.4204635278380479


In [237]:
x = df.drop(["data", "estados", "consumo"], axis=1).copy()

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [238]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.9549585423787639


In [239]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 51061.349763018276
MAE: 32024.049617912762
MAPE: 0.651155608095214

TESTE
Scores obtidos:
RMSE: 88047.15402996687
MAE: 54282.28229742509
MAPE: 0.34291039958174513


In [181]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=256, activation="relu")),
    model.add(tf.keras.layers.Dense(units=128, activation="relu")),
    model.add(tf.keras.layers.Dense(units=64, activation="relu")),
    model.add(tf.keras.layers.Dense(units=32, activation="relu")),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [242]:
x = df.drop(["estados", "data", "consumo", "producao"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 2ms/step - loss: 68131569664.0000 - RMSE: 261020.2500
Epoch 2/500
146/146 [==============================] - 0s 2ms/step - loss: 17626572800.0000 - RMSE: 132765.1094
Epoch 3/500
146/146 [==============================] - 0s 2ms/step - loss: 7076975616.0000 - RMSE: 84124.7656
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 5907714048.0000 - RMSE: 76861.6562
Epoch 5/500
146/146 [==============================] - 0s 2ms/step - loss: 5320099840.0000 - RMSE: 72939.0156
Epoch 6/500
146/146 [==============================] - 0s 2ms/step - loss: 5064832512.0000 - RMSE: 71167.6328
Epoch 7/500
146/146 [==============================] - 0s 3ms/step - loss: 4793539072.0000 - RMSE: 69235.3906
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 4359118848.0000 - RMSE: 66023.6250
Epoch 9/500
146/146 [==============================] - 0s 2ms/step - loss: 4387690496.0000 - RMSE: 66239.6406
Epoch 

146/146 [==============================] - 0s 3ms/step - loss: 1761086336.0000 - RMSE: 41965.3008
Epoch 75/500
146/146 [==============================] - 0s 3ms/step - loss: 1639446656.0000 - RMSE: 40490.0820
Epoch 76/500
146/146 [==============================] - 0s 3ms/step - loss: 1776394624.0000 - RMSE: 42147.2969
Epoch 77/500
146/146 [==============================] - 0s 3ms/step - loss: 1709971328.0000 - RMSE: 41351.8008
Epoch 78/500
146/146 [==============================] - 0s 3ms/step - loss: 1670511232.0000 - RMSE: 40871.8867
Epoch 79/500
146/146 [==============================] - 0s 2ms/step - loss: 1812596352.0000 - RMSE: 42574.5977
Epoch 80/500
146/146 [==============================] - 0s 2ms/step - loss: 1693357824.0000 - RMSE: 41150.4297
Epoch 81/500
146/146 [==============================] - 0s 2ms/step - loss: 1652263424.0000 - RMSE: 40648.0430
Epoch 82/500
146/146 [==============================] - 0s 3ms/step - loss: 1693571584.0000 - RMSE: 41153.0273
Epoch 83/500
1

146/146 [==============================] - 0s 1ms/step - loss: 1512968704.0000 - RMSE: 38896.8984
Epoch 148/500
146/146 [==============================] - 0s 1ms/step - loss: 1552087296.0000 - RMSE: 39396.5391
Epoch 149/500
146/146 [==============================] - 0s 1ms/step - loss: 1664218112.0000 - RMSE: 40794.8281
Epoch 150/500
146/146 [==============================] - 0s 1ms/step - loss: 1625406080.0000 - RMSE: 40316.3242
Epoch 151/500
146/146 [==============================] - 0s 1ms/step - loss: 1658318080.0000 - RMSE: 40722.4531
Epoch 152/500
146/146 [==============================] - 0s 1ms/step - loss: 1603828992.0000 - RMSE: 40047.8320
Epoch 153/500
146/146 [==============================] - 0s 1ms/step - loss: 1589077248.0000 - RMSE: 39863.2305
Epoch 154/500
146/146 [==============================] - 0s 1ms/step - loss: 1566853888.0000 - RMSE: 39583.5039
Epoch 155/500
146/146 [==============================] - 0s 1ms/step - loss: 1588305792.0000 - RMSE: 39853.5547
Epoch 

146/146 [==============================] - 0s 2ms/step - loss: 1458855552.0000 - RMSE: 38194.9688
Epoch 221/500
146/146 [==============================] - 0s 2ms/step - loss: 1447048704.0000 - RMSE: 38040.0938
Epoch 222/500
146/146 [==============================] - 0s 2ms/step - loss: 1440430336.0000 - RMSE: 37953.0000
Epoch 223/500
146/146 [==============================] - 0s 1ms/step - loss: 1560301312.0000 - RMSE: 39500.6484
Epoch 224/500
146/146 [==============================] - 0s 1ms/step - loss: 1436907776.0000 - RMSE: 37906.5664
Epoch 225/500
146/146 [==============================] - 0s 1ms/step - loss: 1461776768.0000 - RMSE: 38233.1875
Epoch 226/500
146/146 [==============================] - 0s 2ms/step - loss: 1604674688.0000 - RMSE: 40058.3906
Epoch 227/500
146/146 [==============================] - 0s 2ms/step - loss: 1551128960.0000 - RMSE: 39384.3750
Epoch 228/500
146/146 [==============================] - 0s 1ms/step - loss: 1566701056.0000 - RMSE: 39581.5742
Epoch 

146/146 [==============================] - 0s 3ms/step - loss: 1401913216.0000 - RMSE: 37442.1328
Epoch 294/500
146/146 [==============================] - 0s 2ms/step - loss: 1478974208.0000 - RMSE: 38457.4336
Epoch 295/500
146/146 [==============================] - 0s 2ms/step - loss: 1352891904.0000 - RMSE: 36781.6797
Epoch 296/500
146/146 [==============================] - 0s 2ms/step - loss: 1425328768.0000 - RMSE: 37753.5273
Epoch 297/500
146/146 [==============================] - 0s 1ms/step - loss: 1398754304.0000 - RMSE: 37399.9258
Epoch 298/500
146/146 [==============================] - 0s 1ms/step - loss: 1424797952.0000 - RMSE: 37746.4961
Epoch 299/500
146/146 [==============================] - 0s 1ms/step - loss: 1361932672.0000 - RMSE: 36904.3711
Epoch 300/500
146/146 [==============================] - 0s 1ms/step - loss: 1319858688.0000 - RMSE: 36329.8594
Epoch 301/500
146/146 [==============================] - 0s 1ms/step - loss: 1409351552.0000 - RMSE: 37541.3320
Epoch 

146/146 [==============================] - 0s 1ms/step - loss: 1249417472.0000 - RMSE: 35347.1016
Epoch 367/500
146/146 [==============================] - 0s 1ms/step - loss: 1305094912.0000 - RMSE: 36126.0977
Epoch 368/500
146/146 [==============================] - 0s 1ms/step - loss: 1346137600.0000 - RMSE: 36689.7461
Epoch 369/500
146/146 [==============================] - 0s 1ms/step - loss: 1327609984.0000 - RMSE: 36436.3828
Epoch 370/500
146/146 [==============================] - 0s 1ms/step - loss: 1292432000.0000 - RMSE: 35950.4102
Epoch 371/500
146/146 [==============================] - 0s 1ms/step - loss: 1309200000.0000 - RMSE: 36182.8672
Epoch 372/500
146/146 [==============================] - 0s 1ms/step - loss: 1240428288.0000 - RMSE: 35219.7148
Epoch 373/500
146/146 [==============================] - 0s 1ms/step - loss: 1260400128.0000 - RMSE: 35502.1133
Epoch 374/500
146/146 [==============================] - 0s 1ms/step - loss: 1243686656.0000 - RMSE: 35265.9414
Epoch 

146/146 [==============================] - 0s 1ms/step - loss: 1230599296.0000 - RMSE: 35079.8984
Epoch 440/500
146/146 [==============================] - 0s 1ms/step - loss: 1129449216.0000 - RMSE: 33607.2773
Epoch 441/500
146/146 [==============================] - 0s 1ms/step - loss: 1241094016.0000 - RMSE: 35229.1641
Epoch 442/500
146/146 [==============================] - 0s 1ms/step - loss: 1163980032.0000 - RMSE: 34117.1523
Epoch 443/500
146/146 [==============================] - 0s 1ms/step - loss: 1180070272.0000 - RMSE: 34352.1523
Epoch 444/500
146/146 [==============================] - 0s 1ms/step - loss: 1292660096.0000 - RMSE: 35953.5820
Epoch 445/500
146/146 [==============================] - 0s 1ms/step - loss: 1167387392.0000 - RMSE: 34167.0508
Epoch 446/500
146/146 [==============================] - 0s 1ms/step - loss: 1237358592.0000 - RMSE: 35176.1094
Epoch 447/500
146/146 [==============================] - 0s 1ms/step - loss: 1156293888.0000 - RMSE: 34004.3203
Epoch 

In [243]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 26303.13542026353
MAE: 15853.766910153734
MAPE: 0.24144434085420163
TESTE
Scores obtidos:
RMSE: 45423.81502263361
MAE: 31136.91863264638
MAPE: 0.5355255019178464


In [ ]:
================
TRAIN
================
Scores obtidos:
RMSE: 22807.097324172653
MAE: 13047.578577826669
MAPE: 0.15626501595083667
================
TESTE
================
Scores obtidos:
RMSE: 38398.31929394046
MAE: 25176.616115288693
MAPE: 0.34854061088685906

In [245]:
x = df.drop(["estados", "data", "consumo"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 1ms/step - loss: 67369742336.0000 - RMSE: 259556.8125
Epoch 2/500
146/146 [==============================] - 0s 1ms/step - loss: 18777323520.0000 - RMSE: 137030.3750
Epoch 3/500
146/146 [==============================] - 0s 1ms/step - loss: 7207237120.0000 - RMSE: 84895.4453
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 6077259776.0000 - RMSE: 77956.7812
Epoch 5/500
146/146 [==============================] - 0s 2ms/step - loss: 5505097216.0000 - RMSE: 74196.3438
Epoch 6/500
146/146 [==============================] - 0s 1ms/step - loss: 5358016512.0000 - RMSE: 73198.4766
Epoch 7/500
146/146 [==============================] - 0s 2ms/step - loss: 4675815936.0000 - RMSE: 68379.9375
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 4355995648.0000 - RMSE: 65999.9688
Epoch 9/500
146/146 [==============================] - 0s 2ms/step - loss: 3921143040.0000 - RMSE: 62619.0312
Epoch 

146/146 [==============================] - 0s 1ms/step - loss: 1692739968.0000 - RMSE: 41142.9219
Epoch 75/500
146/146 [==============================] - 0s 1ms/step - loss: 1649649792.0000 - RMSE: 40615.8828
Epoch 76/500
146/146 [==============================] - 0s 1ms/step - loss: 1708627584.0000 - RMSE: 41335.5469
Epoch 77/500
146/146 [==============================] - 0s 1ms/step - loss: 1614442752.0000 - RMSE: 40180.1289
Epoch 78/500
146/146 [==============================] - 0s 1ms/step - loss: 1729356416.0000 - RMSE: 41585.5312
Epoch 79/500
146/146 [==============================] - 0s 2ms/step - loss: 1739801984.0000 - RMSE: 41710.9336
Epoch 80/500
146/146 [==============================] - 0s 2ms/step - loss: 1580826752.0000 - RMSE: 39759.6133
Epoch 81/500
146/146 [==============================] - 0s 2ms/step - loss: 1820129536.0000 - RMSE: 42662.9766
Epoch 82/500
146/146 [==============================] - 0s 2ms/step - loss: 1680440832.0000 - RMSE: 40993.1797
Epoch 83/500
1

146/146 [==============================] - 0s 2ms/step - loss: 1459920256.0000 - RMSE: 38208.9023
Epoch 148/500
146/146 [==============================] - 0s 2ms/step - loss: 1542243968.0000 - RMSE: 39271.4141
Epoch 149/500
146/146 [==============================] - 0s 2ms/step - loss: 1527285120.0000 - RMSE: 39080.4961
Epoch 150/500
146/146 [==============================] - 0s 1ms/step - loss: 1452175232.0000 - RMSE: 38107.4180
Epoch 151/500
146/146 [==============================] - 0s 2ms/step - loss: 1514053376.0000 - RMSE: 38910.8398
Epoch 152/500
146/146 [==============================] - 0s 2ms/step - loss: 1394848128.0000 - RMSE: 37347.6641
Epoch 153/500
146/146 [==============================] - 0s 2ms/step - loss: 1426484224.0000 - RMSE: 37768.8242
Epoch 154/500
146/146 [==============================] - 0s 2ms/step - loss: 1459834752.0000 - RMSE: 38207.7852
Epoch 155/500
146/146 [==============================] - 0s 1ms/step - loss: 1513862656.0000 - RMSE: 38908.3867
Epoch 

146/146 [==============================] - 0s 2ms/step - loss: 1367450112.0000 - RMSE: 36979.0508
Epoch 221/500
146/146 [==============================] - 0s 2ms/step - loss: 1272713472.0000 - RMSE: 35675.1094
Epoch 222/500
146/146 [==============================] - 0s 2ms/step - loss: 1304596992.0000 - RMSE: 36119.2070
Epoch 223/500
146/146 [==============================] - 0s 1ms/step - loss: 1313016832.0000 - RMSE: 36235.5742
Epoch 224/500
146/146 [==============================] - 0s 1ms/step - loss: 1500966656.0000 - RMSE: 38742.3125
Epoch 225/500
146/146 [==============================] - 0s 1ms/step - loss: 1271217536.0000 - RMSE: 35654.1367
Epoch 226/500
146/146 [==============================] - 0s 2ms/step - loss: 1457743104.0000 - RMSE: 38180.4023
Epoch 227/500
146/146 [==============================] - 0s 1ms/step - loss: 1333995264.0000 - RMSE: 36523.8984
Epoch 228/500
146/146 [==============================] - 0s 1ms/step - loss: 1311155712.0000 - RMSE: 36209.8828
Epoch 

146/146 [==============================] - 0s 1ms/step - loss: 1432581248.0000 - RMSE: 37849.4531
Epoch 294/500
146/146 [==============================] - 0s 1ms/step - loss: 1362855680.0000 - RMSE: 36916.8750
Epoch 295/500
146/146 [==============================] - 0s 1ms/step - loss: 1318302464.0000 - RMSE: 36308.4336
Epoch 296/500
146/146 [==============================] - 0s 1ms/step - loss: 1308411392.0000 - RMSE: 36171.9688
Epoch 297/500
146/146 [==============================] - 0s 1ms/step - loss: 1223970432.0000 - RMSE: 34985.2891
Epoch 298/500
146/146 [==============================] - 0s 1ms/step - loss: 1149606400.0000 - RMSE: 33905.8477
Epoch 299/500
146/146 [==============================] - 0s 1ms/step - loss: 1435144576.0000 - RMSE: 37883.3008
Epoch 300/500
146/146 [==============================] - 0s 1ms/step - loss: 1275335040.0000 - RMSE: 35711.8320
Epoch 301/500
146/146 [==============================] - 0s 1ms/step - loss: 1317361536.0000 - RMSE: 36295.4766
Epoch 

146/146 [==============================] - 0s 1ms/step - loss: 1219208192.0000 - RMSE: 34917.1602
Epoch 367/500
146/146 [==============================] - 0s 1ms/step - loss: 1253839104.0000 - RMSE: 35409.5898
Epoch 368/500
146/146 [==============================] - 0s 1ms/step - loss: 1236912000.0000 - RMSE: 35169.7578
Epoch 369/500
146/146 [==============================] - 0s 1ms/step - loss: 1232503040.0000 - RMSE: 35107.0234
Epoch 370/500
146/146 [==============================] - 0s 1ms/step - loss: 1184905216.0000 - RMSE: 34422.4531
Epoch 371/500
146/146 [==============================] - 0s 1ms/step - loss: 1136795776.0000 - RMSE: 33716.4023
Epoch 372/500
146/146 [==============================] - 0s 1ms/step - loss: 1304684672.0000 - RMSE: 36120.4180
Epoch 373/500
146/146 [==============================] - 0s 1ms/step - loss: 1226111360.0000 - RMSE: 35015.8711
Epoch 374/500
146/146 [==============================] - 0s 1ms/step - loss: 1332377472.0000 - RMSE: 36501.7461
Epoch 

146/146 [==============================] - 0s 1ms/step - loss: 1163180800.0000 - RMSE: 34105.4375
Epoch 440/500
146/146 [==============================] - 0s 1ms/step - loss: 1258282112.0000 - RMSE: 35472.2734
Epoch 441/500
146/146 [==============================] - 0s 1ms/step - loss: 1210002176.0000 - RMSE: 34785.0859
Epoch 442/500
146/146 [==============================] - 0s 1ms/step - loss: 1161813248.0000 - RMSE: 34085.3828
Epoch 443/500
146/146 [==============================] - 0s 1ms/step - loss: 1251078912.0000 - RMSE: 35370.5938
Epoch 444/500
146/146 [==============================] - 0s 1ms/step - loss: 1352306816.0000 - RMSE: 36773.7227
Epoch 445/500
146/146 [==============================] - 0s 1ms/step - loss: 1227073024.0000 - RMSE: 35029.6016
Epoch 446/500
146/146 [==============================] - 0s 1ms/step - loss: 1138804864.0000 - RMSE: 33746.1836
Epoch 447/500
146/146 [==============================] - 0s 1ms/step - loss: 1277039360.0000 - RMSE: 35735.6875
Epoch 

In [246]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 26075.572447140705
MAE: 16196.812356806335
MAPE: 0.2907686991361337
TESTE
Scores obtidos:
RMSE: 27551.9014990335
MAE: 18407.06254399871
MAPE: 0.39691104317960985


In [248]:
x = pd.get_dummies(data=df, columns=["estados"], drop_first=True).drop(["data", "consumo"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 1ms/step - loss: 65195859968.0000 - RMSE: 255334.7969
Epoch 2/500
146/146 [==============================] - 0s 2ms/step - loss: 8525268480.0000 - RMSE: 92332.3828
Epoch 3/500
146/146 [==============================] - 0s 3ms/step - loss: 3250401792.0000 - RMSE: 57012.2969
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 2729658624.0000 - RMSE: 52246.1367
Epoch 5/500
146/146 [==============================] - 0s 2ms/step - loss: 2515936256.0000 - RMSE: 50159.1094
Epoch 6/500
146/146 [==============================] - 0s 2ms/step - loss: 2611849728.0000 - RMSE: 51106.2578
Epoch 7/500
146/146 [==============================] - 0s 2ms/step - loss: 2251320832.0000 - RMSE: 47448.0859
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 2081638528.0000 - RMSE: 45624.9766
Epoch 9/500
146/146 [==============================] - 0s 3ms/step - loss: 1940155264.0000 - RMSE: 44047.1953
Epoch 10

146/146 [==============================] - 0s 2ms/step - loss: 1381197056.0000 - RMSE: 37164.4609
Epoch 75/500
146/146 [==============================] - 0s 2ms/step - loss: 1355089024.0000 - RMSE: 36811.5352
Epoch 76/500
146/146 [==============================] - 0s 2ms/step - loss: 1361301760.0000 - RMSE: 36895.8242
Epoch 77/500
146/146 [==============================] - 0s 3ms/step - loss: 1293787648.0000 - RMSE: 35969.2578
Epoch 78/500
146/146 [==============================] - 0s 1ms/step - loss: 1403882624.0000 - RMSE: 37468.4219
Epoch 79/500
146/146 [==============================] - 0s 1ms/step - loss: 1429646080.0000 - RMSE: 37810.6602
Epoch 80/500
146/146 [==============================] - 0s 1ms/step - loss: 1264522880.0000 - RMSE: 35560.1289
Epoch 81/500
146/146 [==============================] - 0s 1ms/step - loss: 1514857856.0000 - RMSE: 38921.1758
Epoch 82/500
146/146 [==============================] - 0s 1ms/step - loss: 1387660928.0000 - RMSE: 37251.3203
Epoch 83/500
1

146/146 [==============================] - 0s 1ms/step - loss: 1156995968.0000 - RMSE: 34014.6445
Epoch 148/500
146/146 [==============================] - 0s 1ms/step - loss: 1199110528.0000 - RMSE: 34628.1758
Epoch 149/500
146/146 [==============================] - 0s 1ms/step - loss: 1160018944.0000 - RMSE: 34059.0508
Epoch 150/500
146/146 [==============================] - 0s 2ms/step - loss: 1121495424.0000 - RMSE: 33488.7344
Epoch 151/500
146/146 [==============================] - 0s 2ms/step - loss: 1189201664.0000 - RMSE: 34484.8047
Epoch 152/500
146/146 [==============================] - 0s 3ms/step - loss: 1076149632.0000 - RMSE: 32804.7188
Epoch 153/500
146/146 [==============================] - 0s 2ms/step - loss: 1057016512.0000 - RMSE: 32511.7910
Epoch 154/500
146/146 [==============================] - 0s 2ms/step - loss: 1205992960.0000 - RMSE: 34727.4102
Epoch 155/500
146/146 [==============================] - 0s 2ms/step - loss: 1104516096.0000 - RMSE: 33234.2617
Epoch 

146/146 [==============================] - 0s 3ms/step - loss: 1079341312.0000 - RMSE: 32853.3320
Epoch 221/500
146/146 [==============================] - 0s 3ms/step - loss: 1027717056.0000 - RMSE: 32058.0273
Epoch 222/500
146/146 [==============================] - 0s 2ms/step - loss: 983996480.0000 - RMSE: 31368.7188
Epoch 223/500
146/146 [==============================] - 0s 3ms/step - loss: 1010210240.0000 - RMSE: 31783.8047
Epoch 224/500
146/146 [==============================] - 0s 3ms/step - loss: 1158276736.0000 - RMSE: 34033.4648
Epoch 225/500
146/146 [==============================] - 0s 2ms/step - loss: 943068480.0000 - RMSE: 30709.4199
Epoch 226/500
146/146 [==============================] - 0s 3ms/step - loss: 1146630528.0000 - RMSE: 33861.9336
Epoch 227/500
146/146 [==============================] - 0s 2ms/step - loss: 1082632192.0000 - RMSE: 32903.3750
Epoch 228/500
146/146 [==============================] - 0s 3ms/step - loss: 1050400640.0000 - RMSE: 32409.8848
Epoch 22

146/146 [==============================] - 0s 1ms/step - loss: 1175725312.0000 - RMSE: 34288.8516
Epoch 294/500
146/146 [==============================] - 0s 1ms/step - loss: 1087943552.0000 - RMSE: 32983.9883
Epoch 295/500
146/146 [==============================] - 0s 1ms/step - loss: 1047159936.0000 - RMSE: 32359.8516
Epoch 296/500
146/146 [==============================] - 0s 1ms/step - loss: 1052897408.0000 - RMSE: 32448.3809
Epoch 297/500
146/146 [==============================] - 0s 1ms/step - loss: 949349568.0000 - RMSE: 30811.5176
Epoch 298/500
146/146 [==============================] - 0s 1ms/step - loss: 906727936.0000 - RMSE: 30111.9238
Epoch 299/500
146/146 [==============================] - 0s 1ms/step - loss: 1165969920.0000 - RMSE: 34146.3008
Epoch 300/500
146/146 [==============================] - 0s 1ms/step - loss: 1038213248.0000 - RMSE: 32221.3164
Epoch 301/500
146/146 [==============================] - 0s 1ms/step - loss: 1082559616.0000 - RMSE: 32902.2734
Epoch 30

146/146 [==============================] - 0s 1ms/step - loss: 917143936.0000 - RMSE: 30284.3848
Epoch 367/500
146/146 [==============================] - 0s 1ms/step - loss: 981516800.0000 - RMSE: 31329.1680
Epoch 368/500
146/146 [==============================] - 0s 1ms/step - loss: 957747200.0000 - RMSE: 30947.4902
Epoch 369/500
146/146 [==============================] - 0s 993us/step - loss: 1011212544.0000 - RMSE: 31799.5684
Epoch 370/500
146/146 [==============================] - 0s 1ms/step - loss: 951488256.0000 - RMSE: 30846.2031
Epoch 371/500
146/146 [==============================] - 0s 1ms/step - loss: 873301184.0000 - RMSE: 29551.6699
Epoch 372/500
146/146 [==============================] - 0s 1ms/step - loss: 1054162944.0000 - RMSE: 32467.8750
Epoch 373/500
146/146 [==============================] - 0s 1ms/step - loss: 1024652672.0000 - RMSE: 32010.1973
Epoch 374/500
146/146 [==============================] - 0s 1ms/step - loss: 1036174784.0000 - RMSE: 32189.6680
Epoch 375

146/146 [==============================] - 0s 2ms/step - loss: 886565376.0000 - RMSE: 29775.2480
Epoch 440/500
146/146 [==============================] - 0s 3ms/step - loss: 979974848.0000 - RMSE: 31304.5508
Epoch 441/500
146/146 [==============================] - 0s 3ms/step - loss: 937330368.0000 - RMSE: 30615.8516
Epoch 442/500
146/146 [==============================] - 0s 3ms/step - loss: 960712064.0000 - RMSE: 30995.3555
Epoch 443/500
146/146 [==============================] - 0s 2ms/step - loss: 983333568.0000 - RMSE: 31358.1504
Epoch 444/500
146/146 [==============================] - 0s 2ms/step - loss: 1088127360.0000 - RMSE: 32986.7773
Epoch 445/500
146/146 [==============================] - 0s 1ms/step - loss: 994412224.0000 - RMSE: 31534.3027
Epoch 446/500
146/146 [==============================] - 0s 1ms/step - loss: 897265152.0000 - RMSE: 29954.3848
Epoch 447/500
146/146 [==============================] - 0s 1ms/step - loss: 1012008896.0000 - RMSE: 31812.0879
Epoch 448/500

In [249]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 22118.94515508848
MAE: 11347.287327778968
MAPE: 0.10623667394327253
TESTE
Scores obtidos:
RMSE: 25073.523052175206
MAE: 14999.138322926556
MAPE: 0.17874258235578142


In [251]:
x = pd.get_dummies(data=df, columns=["estados"], drop_first=True).drop(["data", "consumo", "producao"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
146/146 [==============================] - 1s 1ms/step - loss: 65459220480.0000 - RMSE: 255850.0000
Epoch 2/500
146/146 [==============================] - 0s 1ms/step - loss: 8927627264.0000 - RMSE: 94486.1250
Epoch 3/500
146/146 [==============================] - 0s 2ms/step - loss: 3292027648.0000 - RMSE: 57376.1953
Epoch 4/500
146/146 [==============================] - 0s 2ms/step - loss: 2770957312.0000 - RMSE: 52639.8828
Epoch 5/500
146/146 [==============================] - 0s 1ms/step - loss: 2537138176.0000 - RMSE: 50370.0117
Epoch 6/500
146/146 [==============================] - 0s 1ms/step - loss: 2642412288.0000 - RMSE: 51404.3984
Epoch 7/500
146/146 [==============================] - 0s 2ms/step - loss: 2251692800.0000 - RMSE: 47452.0039
Epoch 8/500
146/146 [==============================] - 0s 2ms/step - loss: 2097421056.0000 - RMSE: 45797.6094
Epoch 9/500
146/146 [==============================] - 0s 3ms/step - loss: 1952263040.0000 - RMSE: 44184.4219
Epoch 10

146/146 [==============================] - 0s 1ms/step - loss: 1393562112.0000 - RMSE: 37330.4453
Epoch 75/500
146/146 [==============================] - 0s 1ms/step - loss: 1358575744.0000 - RMSE: 36858.8633
Epoch 76/500
146/146 [==============================] - 0s 1ms/step - loss: 1368670976.0000 - RMSE: 36995.5547
Epoch 77/500
146/146 [==============================] - 0s 1ms/step - loss: 1299956224.0000 - RMSE: 36054.9062
Epoch 78/500
146/146 [==============================] - 0s 1ms/step - loss: 1415820928.0000 - RMSE: 37627.3945
Epoch 79/500
146/146 [==============================] - 0s 1ms/step - loss: 1449788032.0000 - RMSE: 38076.0820
Epoch 80/500
146/146 [==============================] - 0s 1ms/step - loss: 1275738752.0000 - RMSE: 35717.4844
Epoch 81/500
146/146 [==============================] - 0s 1ms/step - loss: 1528899328.0000 - RMSE: 39101.1406
Epoch 82/500
146/146 [==============================] - 0s 1ms/step - loss: 1395570176.0000 - RMSE: 37357.3320
Epoch 83/500
1

146/146 [==============================] - 0s 1ms/step - loss: 1159826432.0000 - RMSE: 34056.2227
Epoch 148/500
146/146 [==============================] - 0s 1ms/step - loss: 1189956864.0000 - RMSE: 34495.7500
Epoch 149/500
146/146 [==============================] - 0s 1ms/step - loss: 1153959168.0000 - RMSE: 33969.9727
Epoch 150/500
146/146 [==============================] - 0s 1ms/step - loss: 1159677056.0000 - RMSE: 34054.0312
Epoch 151/500
146/146 [==============================] - 0s 1ms/step - loss: 1221718784.0000 - RMSE: 34953.0938
Epoch 152/500
146/146 [==============================] - 0s 1ms/step - loss: 1080772608.0000 - RMSE: 32875.1055
Epoch 153/500
146/146 [==============================] - 0s 1ms/step - loss: 1066908736.0000 - RMSE: 32663.5684
Epoch 154/500
146/146 [==============================] - 0s 1ms/step - loss: 1185153536.0000 - RMSE: 34426.0586
Epoch 155/500
146/146 [==============================] - 0s 1ms/step - loss: 1138763392.0000 - RMSE: 33745.5703
Epoch 

146/146 [==============================] - 0s 1ms/step - loss: 1091290112.0000 - RMSE: 33034.6797
Epoch 221/500
146/146 [==============================] - 0s 1ms/step - loss: 993986368.0000 - RMSE: 31527.5488
Epoch 222/500
146/146 [==============================] - 0s 1ms/step - loss: 1014755136.0000 - RMSE: 31855.2207
Epoch 223/500
146/146 [==============================] - 0s 1ms/step - loss: 1047329792.0000 - RMSE: 32362.4746
Epoch 224/500
146/146 [==============================] - 0s 1ms/step - loss: 1178539904.0000 - RMSE: 34329.8672
Epoch 225/500
146/146 [==============================] - 0s 2ms/step - loss: 971069952.0000 - RMSE: 31161.9961
Epoch 226/500
146/146 [==============================] - 0s 2ms/step - loss: 1171857792.0000 - RMSE: 34232.4102
Epoch 227/500
146/146 [==============================] - 0s 3ms/step - loss: 1086400768.0000 - RMSE: 32960.5938
Epoch 228/500
146/146 [==============================] - 0s 2ms/step - loss: 1089019520.0000 - RMSE: 33000.2969
Epoch 22

146/146 [==============================] - 0s 1ms/step - loss: 1191582976.0000 - RMSE: 34519.3125
Epoch 294/500
146/146 [==============================] - 0s 1ms/step - loss: 1088888320.0000 - RMSE: 32998.3086
Epoch 295/500
146/146 [==============================] - 0s 1ms/step - loss: 1070063680.0000 - RMSE: 32711.8281
Epoch 296/500
146/146 [==============================] - 0s 1ms/step - loss: 1062975552.0000 - RMSE: 32603.3066
Epoch 297/500
146/146 [==============================] - 0s 1ms/step - loss: 967196096.0000 - RMSE: 31099.7773
Epoch 298/500
146/146 [==============================] - 0s 1ms/step - loss: 908586048.0000 - RMSE: 30142.7617
Epoch 299/500
146/146 [==============================] - 0s 1ms/step - loss: 1178460416.0000 - RMSE: 34328.7109
Epoch 300/500
146/146 [==============================] - 0s 1ms/step - loss: 1036578688.0000 - RMSE: 32195.9414
Epoch 301/500
146/146 [==============================] - 0s 1ms/step - loss: 1061287360.0000 - RMSE: 32577.4062
Epoch 30

146/146 [==============================] - 0s 1ms/step - loss: 1005537088.0000 - RMSE: 31710.2051
Epoch 367/500
146/146 [==============================] - 0s 1ms/step - loss: 1037607040.0000 - RMSE: 32211.9082
Epoch 368/500
146/146 [==============================] - 0s 1ms/step - loss: 999762752.0000 - RMSE: 31619.0254
Epoch 369/500
146/146 [==============================] - 0s 1ms/step - loss: 1035730624.0000 - RMSE: 32182.7695
Epoch 370/500
146/146 [==============================] - 0s 1ms/step - loss: 993179968.0000 - RMSE: 31514.7578
Epoch 371/500
146/146 [==============================] - 0s 1ms/step - loss: 888592128.0000 - RMSE: 29809.2617
Epoch 372/500
146/146 [==============================] - 0s 1ms/step - loss: 1130164736.0000 - RMSE: 33617.9219
Epoch 373/500
146/146 [==============================] - 0s 1ms/step - loss: 1073168704.0000 - RMSE: 32759.2539
Epoch 374/500
146/146 [==============================] - 0s 1ms/step - loss: 1088335616.0000 - RMSE: 32989.9336
Epoch 375

146/146 [==============================] - 0s 1ms/step - loss: 989816256.0000 - RMSE: 31461.3457
Epoch 440/500
146/146 [==============================] - 0s 1ms/step - loss: 1059470208.0000 - RMSE: 32549.5039
Epoch 441/500
146/146 [==============================] - 0s 1ms/step - loss: 979956096.0000 - RMSE: 31304.2500
Epoch 442/500
146/146 [==============================] - 0s 1ms/step - loss: 962578048.0000 - RMSE: 31025.4414
Epoch 443/500
146/146 [==============================] - 0s 1ms/step - loss: 1028288512.0000 - RMSE: 32066.9375
Epoch 444/500
146/146 [==============================] - 0s 1ms/step - loss: 1129180928.0000 - RMSE: 33603.2891
Epoch 445/500
146/146 [==============================] - 0s 1ms/step - loss: 1002684608.0000 - RMSE: 31665.1953
Epoch 446/500
146/146 [==============================] - 0s 1ms/step - loss: 924974464.0000 - RMSE: 30413.3926
Epoch 447/500
146/146 [==============================] - 0s 1ms/step - loss: 1059162368.0000 - RMSE: 32544.7754
Epoch 448/

In [252]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 22842.974044750586
MAE: 11991.906759703725
MAPE: 0.12137462404970259
TESTE
Scores obtidos:
RMSE: 24638.211950348432
MAE: 14591.643175961613
MAPE: 0.18156455930319815


In [266]:
df_copy = df[(df["data"] >= 200402) & (df["data"] <= 200412)].copy()
x = pd.get_dummies(data=df_copy, columns=["estados"], drop_first=True)
train_filter = (x["data"] < 200412)
x_train = x[train_filter].drop(["data", "consumo"], axis=1)
y_train = x[train_filter]["consumo"]

x_test = x[~train_filter].drop(["data", "consumo"], axis=1)
y_test = x[~train_filter]["consumo"]

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

Epoch 1/500
9/9 [==============================] - 0s 2ms/step - loss: 34233198592.0000 - RMSE: 185022.1562
Epoch 2/500
9/9 [==============================] - 0s 4ms/step - loss: 34232410112.0000 - RMSE: 185020.0312
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 34230390784.0000 - RMSE: 185014.5625
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 34225027072.0000 - RMSE: 185000.0781
Epoch 5/500
9/9 [==============================] - 0s 3ms/step - loss: 34210074624.0000 - RMSE: 184959.6562
Epoch 6/500
9/9 [==============================] - 0s 3ms/step - loss: 34181173248.0000 - RMSE: 184881.5156
Epoch 7/500
9/9 [==============================] - 0s 3ms/step - loss: 34119145472.0000 - RMSE: 184713.6875
Epoch 8/500
9/9 [==============================] - 0s 3ms/step - loss: 33995280384.0000 - RMSE: 184378.0938
Epoch 9/500
9/9 [==============================] - 0s 3ms/step - loss: 33740806144.0000 - RMSE: 183686.7031
Epoch 10/500
9/9 [==========

9/9 [==============================] - 0s 3ms/step - loss: 421687136.0000 - RMSE: 20535.0215
Epoch 78/500
9/9 [==============================] - 0s 3ms/step - loss: 715374336.0000 - RMSE: 26746.4824
Epoch 79/500
9/9 [==============================] - 0s 2ms/step - loss: 645726528.0000 - RMSE: 25411.1504
Epoch 80/500
9/9 [==============================] - 0s 3ms/step - loss: 699503808.0000 - RMSE: 26448.1348
Epoch 81/500
9/9 [==============================] - 0s 2ms/step - loss: 399662816.0000 - RMSE: 19991.5684
Epoch 82/500
9/9 [==============================] - 0s 3ms/step - loss: 393921984.0000 - RMSE: 19847.4688
Epoch 83/500
9/9 [==============================] - 0s 3ms/step - loss: 851822464.0000 - RMSE: 29185.9980
Epoch 84/500
9/9 [==============================] - 0s 3ms/step - loss: 668307584.0000 - RMSE: 25851.6465
Epoch 85/500
9/9 [==============================] - 0s 3ms/step - loss: 490704576.0000 - RMSE: 22151.8535
Epoch 86/500
9/9 [==============================] - 0s 4ms/

9/9 [==============================] - 0s 3ms/step - loss: 408542464.0000 - RMSE: 20212.4336
Epoch 154/500
9/9 [==============================] - 0s 3ms/step - loss: 374446144.0000 - RMSE: 19350.6113
Epoch 155/500
9/9 [==============================] - 0s 3ms/step - loss: 402738208.0000 - RMSE: 20068.3379
Epoch 156/500
9/9 [==============================] - 0s 3ms/step - loss: 653248576.0000 - RMSE: 25558.7285
Epoch 157/500
9/9 [==============================] - 0s 3ms/step - loss: 748476992.0000 - RMSE: 27358.3066
Epoch 158/500
9/9 [==============================] - 0s 3ms/step - loss: 426159776.0000 - RMSE: 20643.6367
Epoch 159/500
9/9 [==============================] - 0s 3ms/step - loss: 469945504.0000 - RMSE: 21678.2266
Epoch 160/500
9/9 [==============================] - 0s 3ms/step - loss: 373394112.0000 - RMSE: 19323.4082
Epoch 161/500
9/9 [==============================] - 0s 3ms/step - loss: 725787264.0000 - RMSE: 26940.4395
Epoch 162/500
9/9 [==============================] 

9/9 [==============================] - 0s 3ms/step - loss: 558136448.0000 - RMSE: 23624.9121
Epoch 230/500
9/9 [==============================] - 0s 4ms/step - loss: 572164224.0000 - RMSE: 23919.9551
Epoch 231/500
9/9 [==============================] - 0s 3ms/step - loss: 284082848.0000 - RMSE: 16854.7578
Epoch 232/500
9/9 [==============================] - 0s 3ms/step - loss: 550241984.0000 - RMSE: 23457.2383
Epoch 233/500
9/9 [==============================] - 0s 3ms/step - loss: 481135136.0000 - RMSE: 21934.7930
Epoch 234/500
9/9 [==============================] - 0s 3ms/step - loss: 899673600.0000 - RMSE: 29994.5586
Epoch 235/500
9/9 [==============================] - 0s 3ms/step - loss: 488869824.0000 - RMSE: 22110.4004
Epoch 236/500
9/9 [==============================] - 0s 3ms/step - loss: 597798656.0000 - RMSE: 24449.9219
Epoch 237/500
9/9 [==============================] - 0s 3ms/step - loss: 467922880.0000 - RMSE: 21631.5254
Epoch 238/500
9/9 [==============================] 

9/9 [==============================] - 0s 3ms/step - loss: 497926080.0000 - RMSE: 22314.2578
Epoch 306/500
9/9 [==============================] - 0s 3ms/step - loss: 386799232.0000 - RMSE: 19667.2129
Epoch 307/500
9/9 [==============================] - 0s 3ms/step - loss: 529578336.0000 - RMSE: 23012.5684
Epoch 308/500
9/9 [==============================] - 0s 3ms/step - loss: 436939104.0000 - RMSE: 20903.0879
Epoch 309/500
9/9 [==============================] - 0s 4ms/step - loss: 581055488.0000 - RMSE: 24105.0918
Epoch 310/500
9/9 [==============================] - 0s 2ms/step - loss: 459997984.0000 - RMSE: 21447.5645
Epoch 311/500
9/9 [==============================] - 0s 2ms/step - loss: 797230080.0000 - RMSE: 28235.2637
Epoch 312/500
9/9 [==============================] - 0s 2ms/step - loss: 575999616.0000 - RMSE: 23999.9922
Epoch 313/500
9/9 [==============================] - 0s 3ms/step - loss: 557867328.0000 - RMSE: 23619.2148
Epoch 314/500
9/9 [==============================] 

9/9 [==============================] - 0s 1ms/step - loss: 410692448.0000 - RMSE: 20265.5488
Epoch 382/500
9/9 [==============================] - 0s 2ms/step - loss: 509565088.0000 - RMSE: 22573.5488
Epoch 383/500
9/9 [==============================] - 0s 2ms/step - loss: 327365536.0000 - RMSE: 18093.2461
Epoch 384/500
9/9 [==============================] - 0s 2ms/step - loss: 574463360.0000 - RMSE: 23967.9648
Epoch 385/500
9/9 [==============================] - 0s 2ms/step - loss: 358767648.0000 - RMSE: 18941.1621
Epoch 386/500
9/9 [==============================] - 0s 2ms/step - loss: 343479712.0000 - RMSE: 18533.2051
Epoch 387/500
9/9 [==============================] - 0s 2ms/step - loss: 446772576.0000 - RMSE: 21136.9961
Epoch 388/500
9/9 [==============================] - 0s 2ms/step - loss: 551853120.0000 - RMSE: 23491.5547
Epoch 389/500
9/9 [==============================] - 0s 2ms/step - loss: 612008896.0000 - RMSE: 24738.8145
Epoch 390/500
9/9 [==============================] 

9/9 [==============================] - 0s 2ms/step - loss: 535532640.0000 - RMSE: 23141.5781
Epoch 458/500
9/9 [==============================] - 0s 2ms/step - loss: 612731648.0000 - RMSE: 24753.4160
Epoch 459/500
9/9 [==============================] - 0s 2ms/step - loss: 768779264.0000 - RMSE: 27726.8691
Epoch 460/500
9/9 [==============================] - 0s 1ms/step - loss: 388310208.0000 - RMSE: 19705.5879
Epoch 461/500
9/9 [==============================] - 0s 1ms/step - loss: 435119424.0000 - RMSE: 20859.5156
Epoch 462/500
9/9 [==============================] - 0s 1ms/step - loss: 333669312.0000 - RMSE: 18266.6172
Epoch 463/500
9/9 [==============================] - 0s 1ms/step - loss: 784201728.0000 - RMSE: 28003.6016
Epoch 464/500
9/9 [==============================] - 0s 1ms/step - loss: 640352256.0000 - RMSE: 25305.1816
Epoch 465/500
9/9 [==============================] - 0s 1ms/step - loss: 265259328.0000 - RMSE: 16286.7842
Epoch 466/500
9/9 [==============================] 

In [267]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 11365.361984920366
MAE: 5737.867214098668
MAPE: 0.0920544299785111
TESTE
Scores obtidos:
RMSE: 20251.455116795565
MAE: 16771.309583912036
MAPE: 0.6147865217346026


In [ ]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=32, activation="relu", return_sequences=True)),
    model.add(tf.keras.layers.GlobalAveragePooling1D()),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [ ]:
x, y = split_sequences(df.drop(["estados", "data", "consumo"], axis=1).values, 15)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=500, verbose=verbose)

In [ ]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

In [ ]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=128, activation="relu", return_sequences=True)),
    model.add(tf.keras.layers.GlobalAveragePooling1D()),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=64, activation="relu")),
    model.add(tf.keras.layers.Dense(units=32, activation="relu")),
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [ ]:
x, y = split_sequences(df.drop(["estados", "data"], axis=1).values, timesteps)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

In [ ]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

In [ ]:
x, y = split_sequences(df.drop(["estados", "data"], axis=1).values, 7)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

In [ ]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)